# Coverage Monitor

Problem statement

Approach

Who are we?

## Constants

In [4]:
import os

# benchmark directory
benchmark_loc = "/Users/IceCream/Documents/hw22_fuzzing/benchmark"

# studied project
proj = "isort_setting3"
proj_loc = os.path.join(benchmark_loc, proj)

# output directory
out_loc = os.path.join(proj_loc, "analysis")
if not os.path.exists(out_loc):
    os.makedirs(out_loc)

## Lauching AFL

In [ ]:
# afl driver goes here

## Monitoring executions

In [30]:
%run cov_tracker.ipynb

'''
Process code coverage and store the output into cov.csv and exec_c.csv for coverage and execution counts 
["cid", "file", "line", "fn"]
'''
def process_cov(proj_loc, out_loc):
    get_test_cov(proj_loc)
#     pd_cov = list(set([line for k, v in cov.items() for line in v.keys()]))
#     pd_cov = pd.DataFrame([pd_cov], columns=["cid", "line"])
#     display(pd_cov)

def process_info_cov(lcov_loc):
    # info_cov is used to compute the coverage of the first test case, because the da and fnda values are cumulative
    info_cov = {}
    # for initial info trace file
    info_trace_file = os.path.join(lcov_loc, 'trace.lcov_info_final')
    df_info_exec, df_fn = get_cov_info_trace(info_trace_file)
    return df_info_exec, df_fn

'''
Process the coverage backward by subtracting the last trace_id by the before last one to compute the diff
'''
def diff_cov(row, df_exec_c, df_info_exec):
    exec_c, trace_id, file, line = row['exec'], row['trace_id'], row['file'], row['line']
    # select next_row to compute the diff
    if trace_id == 0:
        next_row = df_info_exec[(df_info_exec['file'] == file) & (df_info_exec['line'] == line)]
    else:
        next_row = df_exec_c[(df_exec_c["trace_id"] == trace_id-1) & (df_exec_c['file'] == file) & (df_exec_c['line'] == line)]
    
    if next_row.empty:
        return row
    else:
        row['exec'] = row['exec'] - next_row.iloc[0]['exec']
    return row

'''
Compute the code coverage for each source file, store in df_exec_c
'''
def process_per_sf_cov(trace_id, test_file):
    global df_exec_c
    
    # build matcher object for coverage string
    cov_str = read_file_as_str(test_file)

    # for file
    sf_str_list = cov_str.split("end_of_record")
    for cov_str in sf_str_list[:-1]:
        m = Matcher(cov_str)
        # skip if the instrumented lines is 0
        if(get_lh(m) == 0):
            continue

        # find the source file and use the relative path
        sf = os.path.relpath(get_sf(m), proj_loc)
        # compute coverage
        df_da = get_df_da(m)
        df_da['trace_id'] = trace_id
        df_da['file'] = sf
        df_da = df_da[df_da['exec'] != 0]
        df_exec_c.append(df_da)

'''
Get code coverage from test cases/inputs generated by AFL
'''
def get_test_cov(proj_loc):
    # todo: use dataframes
    global df_exec_c
    lcov_loc = os.path.join(proj_loc, 'isort-seeds-out', 'cov', 'lcov')
    test_files = glob.glob(lcov_loc+'/*_trace.lcov_info_final')
    id_dict = {get_trace_id(test_file): test_file for test_file in test_files}
    
    # collect coverage for info file
    df_info_exec, df_fn = process_info_cov(lcov_loc)
    
    # collect coverage for all other trace_id
    for trace_id in sorted(id_dict):
        test_file = id_dict[trace_id]
        process_per_sf_cov(trace_id, test_file)
         
    # combine all df into one single df
    df_exec_c = pd.concat(df_exec_c)
    
    # sorted reverse, start from the last trace_id
    # ignore last trace_id as it may be incomplete
    df_exec_c = df_exec_c.sort_values(by=['trace_id'], ascending=False)
    df_exec_c = df_exec_c.apply(lambda row: diff_cov(row, df_exec_c, df_info_exec), axis=1)
    
    #print("Test case {}: {}/{}".format(trace_id, len([k for k, v in cov[trace_id].items() if v!=0]), len(cov[trace_id])))
    #print("Initial info trace: {} out of {} lines were covered".format(len([k for k, v in info_cov.items() if v!=0]), len(info_cov)))
    #print("{} functions".format(len(fn_dict)))
    
df_exec_c = []
# set global cov dict
process_cov(proj_loc, out_loc)

# df_ = pd.DataFrame()
# df_.to_csv(file_name)

# output two files, coverage.csv and exec_count.csv

KeyError: 'file'